In [148]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [149]:
%matplotlib inline

In [150]:
df = pd.read_csv("orders_summary.csv")


In [151]:
df.head()

,transaction_id,total_bill_amount,email,transaction_date,quantity,time_spent_min,name,phone,loyalty_enrolled,loyalty_id,tier,points_earned,gender,age,birthday,occupation
0,T000003,55.45,aiden.lam185@hotmail.com,28-Oct-24,5,115,Aiden Lam,1-365-193-6905,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T000009,28.85,alexander.taylor405@gmail.com,28-Oct-24,5,118,Alexander Taylor,1-647-596-8619,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T000025,54.95,sofia.johnson337@hotmail.com,30-Oct-24,5,75,Sofia Johnson,1-416-144-4405,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T000076,57.96,jackson.liu3@outlook.com,3-Nov-24,5,68,Jackson Liu,1-416-759-3586,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T000098,76.98,victoria.martin295@yahoo.com,6-Nov-24,5,110,Victoria Martin,1-289-873-8815,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
df.columns

Index(['transaction_id', 'total_bill_amount', 'email', 'transaction_date',
       'quantity', 'time_spent_min', 'name', 'phone', 'loyalty_enrolled',
       'loyalty_id', 'tier', 'points_earned', 'gender', 'age', 'birthday',
       'occupation'],
      dtype='object')

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     4500 non-null   object 
 1   total_bill_amount  4500 non-null   float64
 2   email              4500 non-null   object 
 3   transaction_date   4500 non-null   object 
 4   quantity           4500 non-null   int64  
 5   time_spent_min     4500 non-null   int64  
 6   name               4500 non-null   object 
 7   phone              4500 non-null   object 
 8   loyalty_enrolled   4500 non-null   object 
 9   loyalty_id         2791 non-null   object 
 10  tier               2791 non-null   object 
 11  points_earned      2791 non-null   float64
 12  gender             2791 non-null   object 
 13  age                2791 non-null   float64
 14  birthday           2791 non-null   object 
 15  occupation         2791 non-null   object 
dtypes: float64(3), int64(2),

In [154]:
drop_cols = [
    'transaction_id',
    'name',
    'phone',
    'loyalty_id',
    'birthday'
]

df = df.drop(columns=drop_cols)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   total_bill_amount  4500 non-null   float64
 1   email              4500 non-null   object 
 2   transaction_date   4500 non-null   object 
 3   quantity           4500 non-null   int64  
 4   time_spent_min     4500 non-null   int64  
 5   loyalty_enrolled   4500 non-null   object 
 6   tier               2791 non-null   object 
 7   points_earned      2791 non-null   float64
 8   gender             2791 non-null   object 
 9   age                2791 non-null   float64
 10  occupation         2791 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 386.8+ KB


In [155]:
# dealing with missing values
df.isnull().sum()

total_bill_amount       0
email                   0
transaction_date        0
quantity                0
time_spent_min          0
loyalty_enrolled        0
tier                 1709
points_earned        1709
gender               1709
age                  1709
occupation           1709
dtype: int64

In [156]:
df['tier'] = np.where(df['loyalty_enrolled'] == "Y",
                      df['tier'].fillna('Unknown'),
                      'Non-member')

df['points_earned'] = np.where(df['loyalty_enrolled'] == "Y",
                               df['points_earned'].fillna(0),
                               0)

df['gender'] = np.where(df['loyalty_enrolled'] == "Y",
                        df['gender'].fillna('Unknown'),
                        'Unknown')

df['age'] = np.where(df['loyalty_enrolled'] == "Y",
                     df['age'].fillna(df['age'].median()),
                     0)  

df['occupation'] = np.where(df['loyalty_enrolled'] == "Y",
                            df['occupation'].fillna('Unknown'),
                            'Unknown')

df = df.drop(columns=['loyalty_enrolled'])




In [157]:
df.sample(5)

,total_bill_amount,email,transaction_date,quantity,time_spent_min,tier,points_earned,gender,age,occupation
485,43.48,henry.brown105@yahoo.com,19-Jul-25,3,137,Non-member,0.0,Unknown,0.0,Unknown
4177,116.52,mila.huang59@hotmail.com,26-Jul-25,10,113,Frequent,7471.0,Male,40.0,Data Analyst
4146,206.39,evelyn.martinez130@hotmail.com,6-Jul-25,13,139,Frequent,7936.0,Male,18.0,Nurse
2178,73.45,luke.lau347@gmail.com,24-Aug-25,6,129,Very Frequent,11361.0,Female,30.0,HR Coordinator
927,24.78,logan.wilson333@yahoo.com,18-Jun-25,2,69,Non-member,0.0,Unknown,0.0,Unknown


In [158]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   total_bill_amount  4500 non-null   float64       
 1   email              4500 non-null   object        
 2   transaction_date   4500 non-null   datetime64[ns]
 3   quantity           4500 non-null   int64         
 4   time_spent_min     4500 non-null   int64         
 5   tier               4500 non-null   object        
 6   points_earned      4500 non-null   float64       
 7   gender             4500 non-null   object        
 8   age                4500 non-null   float64       
 9   occupation         4500 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 351.7+ KB


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26820\3633412906.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['transaction_date'] = pd.to_datetime(df['transaction_date'])


In [159]:
print("Range:", df['transaction_date'].min(), "→", df['transaction_date'].max())

Range: 2024-10-28 00:00:00 → 2025-10-28 00:00:00


In [160]:
import pandas as pd
import numpy as np

def make_churn_dataset_simple(df, snapshot_dates, obs_days=90, churn_days=60):
    """
    df: 清洗好的数据（包含 email, transaction_date, total_bill_amount, time_spent_min 等）
    snapshot_dates: 一组 snapshot 日期（字符串或 Timestamp 都可以）
    obs_days: 观察窗口长度（过去多少天算特征）
    churn_days: 未来窗口长度（多少天内无消费算 churn）
    """
    df = df.copy()
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    
    all_samples = []

    # 确保 snapshot_dates 是 Timestamp 列表
    snapshot_dates = [pd.to_datetime(d) for d in snapshot_dates]

    for snap in snapshot_dates:
        print(f"Deal with snapshot: {snap.date()}")

        # 1️⃣ 观察窗口：snap 往前 obs_days 天
        obs_start = snap - pd.Timedelta(days=obs_days)
        obs_end   = snap
        obs_data = df[(df['transaction_date'] > obs_start) &
                      (df['transaction_date'] <= obs_end)]

        # 若这段时间没人来，直接跳过
        if obs_data.empty:
            print("No data in observation window, skip.")
            continue

        # 2️⃣ 未来窗口：snap 往后 churn_days 天
        fut_start = snap
        fut_end   = snap + pd.Timedelta(days=churn_days)
        fut_data = df[(df['transaction_date'] > fut_start) &
                      (df['transaction_date'] <= fut_end)]

        # 3️⃣ 在观察窗口里算 RFM 特征
        today = snap + pd.Timedelta(days=1)
        rfm = obs_data.groupby('email').agg({
            'transaction_date': lambda x: (today - x.max()).days,  # 距离最近一次消费几天
            'total_bill_amount': ['count', 'sum', 'mean'],         # 次数/总消费/均值
            'time_spent_min': 'mean'                               # 平均停留时间
        }).reset_index()

        rfm.columns = [
            'email',
            'recency_days',
            'frequency',
            'total_spent',
            'avg_bill',
            'avg_time_spent'
        ]

        # 4️⃣ 取 snapshot 之前最新一条 loyalty + demo 信息
        # 即：那一天之前，这个顾客最后一次出现时的 tier、points、gender、age 等
        demo_cols = [
            'email',
            'tier',
            'points_earned',
            'gender',
            'age',
            'occupation',
        ]
        demo_before = df[df['transaction_date'] <= snap][demo_cols + ['transaction_date']]
        demo_before = demo_before.sort_values('transaction_date')
        demo_latest = demo_before.drop_duplicates('email', keep='last').drop(columns='transaction_date')

        # 合并行为特征 + 人口/loyalty 特征
        sample = rfm.merge(demo_latest, on='email', how='left')

        # 5️⃣ 定义 churn：未来窗口内有没有再次出现
        future_emails = set(fut_data['email'].unique())
        sample['churn'] = sample['email'].apply(lambda e: 0 if e in future_emails else 1)

        # 记录这个 sample 是哪个 snapshot 算出来的
        sample['snapshot_date'] = snap

        all_samples.append(sample)

    # 合并所有 snapshot 的样本
    training_data = pd.concat(all_samples, ignore_index=True)
    return training_data


In [161]:
start_date = pd.Timestamp('2024-10-28')
end_date   = pd.Timestamp('2025-10-28')

obs_days = 90      # 观察窗口
churn_days = 60    # 预测窗口
gap_days = 90      # snapshot 间隔（每3个月一个）

# 自动生成 snapshot 列表
snapshots = []
snap_date = start_date + pd.Timedelta(days=obs_days)

while snap_date + pd.Timedelta(days=churn_days) < end_date:
    snapshots.append(snap_date)
    snap_date += pd.Timedelta(days=gap_days)

print("📅 自动生成的 snapshot 日期：")
for s in snapshots:
    print(" -", s.date())

📅 自动生成的 snapshot 日期：
 - 2025-01-26
 - 2025-04-26
 - 2025-07-25


In [162]:
training_data = make_churn_dataset_simple(
    df,
    snapshot_dates=snapshots,
    obs_days=90,
    churn_days=60
)


Deal with snapshot: 2025-01-26
Deal with snapshot: 2025-04-26
Deal with snapshot: 2025-07-25


In [163]:
# 按时间顺序排序
training_data = training_data.sort_values('snapshot_date')

# 最后一组 snapshot 作为测试集
last_snapshot = training_data['snapshot_date'].max()

train_df = training_data[training_data['snapshot_date'] < last_snapshot]
test_df  = training_data[training_data['snapshot_date'] == last_snapshot]

print("训练集样本数：", train_df.shape[0])
print("测试集样本数：", test_df.shape[0])

训练集样本数： 866
测试集样本数： 464


In [164]:
train_df.head()

,email,recency_days,frequency,total_spent,avg_bill,avg_time_spent,tier,points_earned,gender,age,occupation,churn,snapshot_date
0,abigail.chen229@hotmail.com,14,1,101.93,101.930,30.0,Frequent,4022.0,Male,29.0,Nurse,1,2025-01-26
302,mia.lin430@hotmail.com,36,2,184.74,92.370,77.5,Non-member,0.0,Unknown,0.0,Unknown,0,2025-01-26
301,mia.lee54@gmail.com,47,2,329.27,164.635,124.0,Non-member,0.0,Unknown,0.0,Unknown,0,2025-01-26
300,mia.lam354@outlook.com,2,2,258.78,129.390,50.5,Very Frequent,8226.0,Female,54.0,Consultant,0,2025-01-26
299,mia.lam18@hotmail.com,29,5,300.27,60.054,105.2,Non-member,0.0,Unknown,0.0,Unknown,0,2025-01-26


In [165]:
from sklearn.model_selection import train_test_split

# 训练集 & 测试集已经按 snapshot_date 分好了
# train_df, test_df 来自你前面那一步

# 特征列：去掉 email、churn、snapshot_date
drop_cols = ['email', 'churn', 'snapshot_date']

X_train = train_df.drop(columns=drop_cols, errors='ignore')
y_train = train_df['churn']

X_test  = test_df.drop(columns=drop_cols, errors='ignore')
y_test  = test_df['churn']

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (866, 10)
X_test shape: (464, 10)


In [166]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# 数值特征（你可以按实际列名微调）
num_cols = [
    'recency_days',      # 最近一次来店距今多少天
    'frequency',         # 过去窗口内的消费次数
    'total_spent',       # 总消费金额
    'avg_bill',          # 平均客单价
    'avg_time_spent',    # 平均停留时长
    'points_earned',     # 积分
    'age',               # 年龄
]

# 类别特征
cat_cols = [
    'tier',              # 会员等级 / Non-member
    'gender', 
    'occupation'
]

# 特征可能会有缺列，这里做个交集保护一下
num_cols = [c for c in num_cols if c in X_train.columns]
cat_cols = [c for c in cat_cols if c in X_train.columns]

print("数值特征:", num_cols)
print("类别特征:", cat_cols)

# 预处理：数值原样传入，类别做 one-hot
preprocess = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)

# 随机森林模型（简单参数，方便先跑通）
clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    random_state=42,
    class_weight='balanced'   # 类别不平衡时更稳一点
)

# 拼成一个 pipeline
model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('clf', clf)
])

# 训练
model.fit(X_train, y_train)


数值特征: ['recency_days', 'frequency', 'total_spent', 'avg_bill', 'avg_time_spent', 'points_earned', 'age']
类别特征: ['tier', 'gender', 'occupation']


,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [167]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# 预测概率（用来算 AUC）
y_proba = model.predict_proba(X_test)[:, 1]
y_pred  = model.predict(X_test)

print("ROC-AUC 分数：", roc_auc_score(y_test, y_proba))
print("\n分类报告（precision / recall / f1）：\n")
print(classification_report(y_test, y_pred))

print("混淆矩阵：")
print(confusion_matrix(y_test, y_pred))


ROC-AUC 分数： 0.6296333200478277

分类报告（precision / recall / f1）：

              precision    recall  f1-score   support

           0       0.87      0.71      0.78       386
           1       0.24      0.46      0.32        78

    accuracy                           0.67       464
   macro avg       0.55      0.58      0.55       464
weighted avg       0.76      0.67      0.70       464

混淆矩阵：
[[273 113]
 [ 42  36]]


In [168]:
# 从 pipeline 里取出真正的 RF 模型
rf = model.named_steps['clf']
ohe = model.named_steps['preprocess'].named_transformers_['cat']

# one-hot 展开的类别特征名
cat_feature_names = ohe.get_feature_names_out(cat_cols)
feature_names = num_cols + list(cat_feature_names)

import pandas as pd

fi = pd.DataFrame({
    'feature': feature_names,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print(fi.head(20))


                          feature  importance
5                   points_earned    0.148539
4                  avg_time_spent    0.122426
3                        avg_bill    0.119802
2                     total_spent    0.111475
0                    recency_days    0.106188
8                   tier_Frequent    0.069063
6                             age    0.050284
1                       frequency    0.044524
26               occupation_Nurse    0.034357
10             tier_Very Frequent    0.028708
14       gender_Prefer not to say    0.011652
15                 gender_Unknown    0.010303
12                    gender_Male    0.010187
36             occupation_Unknown    0.009729
19          occupation_Consultant    0.009602
29     occupation_Project Manager    0.009542
27  occupation_Operations Manager    0.008684
9                 tier_Non-member    0.008505
31     occupation_Sales Associate    0.007862
17             occupation_Barista    0.007853


In [170]:
from sklearn.model_selection import GridSearchCV

# 1️⃣ 定义要调的参数（注意前面有 clf__）
param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__max_depth': [4, 6, 8, None],
    'clf__min_samples_leaf': [1, 2, 5]
}

# 2️⃣ 把整个 pipeline 丢进 GridSearchCV
grid = GridSearchCV(
    estimator=model,      # 这里直接用 pipeline
    param_grid=param_grid,
    scoring='roc_auc',    # 用 AUC 来评估
    cv=3,                 # 3 折交叉验证
    n_jobs=1              # 建议先写 1，避免你之前的并行报错
)

# 3️⃣ 在训练集上搜索最优参数
grid.fit(X_train, y_train)

print("最佳参数：", grid.best_params_)
print("CV 最佳 AUC：", grid.best_score_)

# 4️⃣ 取出带最优参数的最终模型
best_model = grid.best_estimator_


最佳参数： {'clf__max_depth': 4, 'clf__min_samples_leaf': 2, 'clf__n_estimators': 300}
CV 最佳 AUC： 0.6058639948359574


In [171]:
from sklearn.metrics import roc_auc_score, classification_report

y_proba = best_model.predict_proba(X_test)[:, 1]
y_pred  = best_model.predict(X_test)

print("Test ROC-AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


Test ROC-AUC: 0.6210641689916301
              precision    recall  f1-score   support

           0       0.86      0.68      0.76       386
           1       0.23      0.46      0.31        78

    accuracy                           0.65       464
   macro avg       0.55      0.57      0.53       464
weighted avg       0.76      0.65      0.69       464

